In [ ]:
from pyspark.sql import SQLContext, SparkSession, Window, Row
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

r_fraud_annotations             = spark.table('annotations.cred_annotations')
tab_desenrola = (r_fraud_annotations
                 .select('*')
                 .filter(col('annotation_description').isin('flag_desenrola_materialidade'))
                 .withColumnRenamed('document', 'cpf')
                 .limit(10)
                 .select('*', lit(1).alias('is_user_desenrola'))
)
tab_desenrola.toPandas()

In [ ]:
from datetime import datetime, timedelta

dataCriacaoConta = "2024-02-26T10:57:00.888Z"
dataCriacaoContaDate = datetime.strptime(dataCriacaoConta,"%Y-%m-%dT%H:%M:%S.%fZ")

d = dataCriacaoContaDate - timedelta(days=61)

print(d)

In [ ]:
import os

# Me mostre o caminho absoluto para a pasta arquivos dessa pasta
folder_path = os.path.abspath('arquivos')
folder_path = str(folder_path)

print(folder_path)

In [ ]:
import json
pasta = "/Workspace/Users/kaue.ongaro@picpaybank.com/2 - Homologação/Concessao de Cartoes - Online/"
file_name = "arquivos/payload_inicial.txt"
file_path = pasta + file_name
with open(file_path,'r') as f:
    x = f.read()
    jsonzada = json.loads(x)


In [ ]:
jsonzada

In [ ]:
jsonzada['idSolicitacao']

In [ ]:
def execucaoModular (payload_entrada, modulo):
    
    lista_payloads = payload_entrada

    lista_resultado = []

    for caso in lista_payloads:
        payload = json.load(caso) 
        payload = modulo(payload)
        lista_resultado.append(payload)

    return lista_resultado

In [ ]:
from datetime import datetime
from datetime import date
import pandas as pd

hojeDate = pd.Timestamp(date.today()) # Avaliar como pegar o dia da execução, pois teremos problemas no Pós acomp pois talvez a data da execução não será a mesma na qualavaliaremos a log
dataCriacaoConta = "2024-02-26T17:18:43.888Z"
######################################################################################################################
if dataCriacaoConta == "":
    flagClienteExistente = False
else:
    dataCriacaoContaDate = datetime.strptime(dataCriacaoConta,"%Y-%m-%dT%H:%M:%S.%fZ")
    delta = hojeDate - dataCriacaoContaDate
    print(delta)
    if delta.days > 60:
        flagClienteExistente = True
    else:
        flagClienteExistente = False
print(flagClienteExistente)

In [ ]:
dbutils.library.restartPython()

In [ ]:
import sherlock

In [ ]:
import pandas as pd
# Import the csv file into a pandas dataframe
pasta  = "/Workspace/Users/kaue.ongaro@picpaybank.com/2 - Homologação/Concessao de Cartoes - Online/arquivos/tabelasHomologacao/"
#
arquivo = "Ja_Cliente_Segmentacao_codigo_motor.csv"
file_path = pasta + arquivo
pd.set_option('display.max_rows', 5)
tabela = pd.read_csv(file_path)
tabela

In [ ]:
rangeScoreInterno = "R1"
segmentacaoPolitica = "P1 - Funcionarios"
faixaRendaBruta = "a. < 4k"
flagFuncionario = 1
cpf5e6Digito = 99

# Faixas de CPF
if (0 <= cpf5e6Digito <= 44):
    faixaCPF = "0 <= ..<= 44"
if (45 <= cpf5e6Digito <= 89):
    faixaCPF = "45 <= ..<= 89"
if (90 <= cpf5e6Digito <= 99):
    faixaCPF = "90 <= ..<= 99"

# Primeiro Filtro - Nível rangeScoreInterno
filtroNivelRangeScoreInterno = tabela.loc[(tabela['rangeScoreInterno'] == rangeScoreInterno)]
if filtroNivelRangeScoreInterno.empty:
    filtroNivelRangeScoreInterno = tabela.loc[(tabela['rangeScoreInterno'] == "QUALQUER")]

# Segundo Filtro - Nível segmentacaoPolitica
filtroNivelSegmentacaoPolitica = filtroNivelRangeScoreInterno.loc[(filtroNivelRangeScoreInterno['segmentacaoPolitica'] == segmentacaoPolitica)]
if filtroNivelSegmentacaoPolitica.empty:
    filtroNivelSegmentacaoPolitica = filtroNivelRangeScoreInterno.loc[(filtroNivelRangeScoreInterno['segmentacaoPolitica'] == "QUALQUER")]

# Terceiro Filtro - Nível faixaRendaBruta
filtroNivelFaixaRendaBruta = filtroNivelSegmentacaoPolitica.loc[(filtroNivelSegmentacaoPolitica['faixaRendaBruta'] == faixaRendaBruta)]
if filtroNivelFaixaRendaBruta.empty:
    filtroNivelFaixaRendaBruta = filtroNivelSegmentacaoPolitica.loc[(filtroNivelSegmentacaoPolitica['faixaRendaBruta'] == "QUALQUER")]

# Quarto Filtro - Nível flagFuncionario
filtroNivelflagFuncionario = filtroNivelFaixaRendaBruta.loc[(filtroNivelFaixaRendaBruta['flagFuncionario'] == flagFuncionario)]
if filtroNivelflagFuncionario.empty:
    filtroNivelflagFuncionario = filtroNivelFaixaRendaBruta.loc[(filtroNivelFaixaRendaBruta['flagFuncionario'] == "QUALQUER")]

# Quinto Filtro - Nível 5e6 Digito CPF
filtroNivelCPF = filtroNivelflagFuncionario.loc[(filtroNivelflagFuncionario['cpf5e6Digito'] == faixaCPF)]
if filtroNivelCPF.empty:
    filtroNivelCPF = filtroNivelflagFuncionario.loc[(filtroNivelflagFuncionario['cpf5e6Digito'] == "QUALQUER")]

linha_tab = filtroNivelCPF
# Teste para ver se após filtrar na tabela o cliente foi encontrado
if linha_tab.empty:
    segmentacao = "-1"
    alavancagem = -1
    limiteTeto = -1
    limitePiso = -1
else:
    segmentacao = linha_tab.iloc[0]['SEGMENTAÇÃO FINAL']
    alavancagem = linha_tab.iloc[0]['alavancagem']
    limiteTeto = linha_tab.iloc[0]['limiteTeto']
    limitePiso = linha_tab.iloc[0]['limitePiso']   

#print(segmentacao, alavancagem, limiteTeto, limitePiso)

In [ ]:
filtroNivelflagFuncionario

In [ ]:
filtroNivelCPF

In [ ]:
import pandas as pd
from datetime import datetime
from datetime import date
dataAdmissao = "2002-11-05T17:18:43.888Z"
hojeDate = pd.Timestamp(date.today())  # Avaliar como pegar o dia da execução, pois teremos problemas no Pós acomp pois talvez a data da execução nãoserá a mesma na qual avaliaremos a log
######################################################################################################################

admissaoDate = datetime.strptime(dataAdmissao, "%Y-%m-%dT%H:%M:%S.%fZ")
print("print 1: ",admissaoDate)
admissaoMeses = (hojeDate.year - admissaoDate.year) * 12 + (hojeDate.month - admissaoDate.month)
print("print 2: ",(hojeDate.year - admissaoDate.year) * 12)
print("print 3: ",(hojeDate.month - admissaoDate.month))



print(admissaoMeses)

In [ ]:
dataAdmissao = "2002-11-05T17:18:43.888Z"
hojeDate = pd.Timestamp(date.today())
admissaoDate = datetime.strptime(dataAdmissao, "%Y-%m-%dT%H:%M:%S.%fZ")
admissaoMeses = (hojeDate.year - admissaoDate.year) * 12 + (hojeDate.month - admissaoDate.month)
admissaoMeses

In [ ]:
# GERANDO DATA DIFS PARA ADICIONAR NA MASSA

from datetime import datetime, timedelta

today = datetime.today()
target_date = today - timedelta(days=60)
dif60 = target_date.strftime("%Y-%m-%dT%H:%M:%S.%fZ")

target_date = today - timedelta(days=59)
dif59 = target_date.strftime("%Y-%m-%dT%H:%M:%S.%fZ")

target_date = today - timedelta(days=61)
dif61 = target_date.strftime("%Y-%m-%dT%H:%M:%S.%fZ")

print("data de 59 dias :",dif59)
print("data de 60 dias :",dif60)
print("data de 61 dias :",dif61)

In [ ]:
# como splitar uma massa em partes iguais

#import numpy as np
#dataframeGrande = {}
#num_dataframes = 3
#
#split_dataframes = np.array_split(dataframeGrande, num_dataframes)
#
#for i, df in enumerate(split_dataframes):
#    df.to_csv(f'dataframe{i}.csv', index=False)

In [ ]:
def round_with_steps(number):
    return round(number/50) * 50

round_with_steps(47.33)

In [ ]:
today = date.today()

dataCriacaoConta = '2024-01-17T18:35:34.748956Z'
dataCriacaoConta = datetime.datetime.strptime(dataCriacaoConta[:10], "%Y-%m-%d").date()

type(today)

In [ ]:
delta = today - dataCriacaoConta
delta.days

In [ ]:
import pandas as pd

data = spark.read.parquet("s3://picpay-credit-laboratory/modeling/income/3v5/income3v5_backtest")
distinct_df = data.select("source_income").distinct().toPandas()
distinct_df

# Teste funcao addListaLogs

In [10]:
import json

jsonAnalise = '''
{
    "marcacaoMesa": "N",
    "etapa": "FILTROS POLITICA",
    "flagErroBacen": false,
    "flagErroSerasa": false,
    "flagErroAurora": false,
    "flagErroVisaoCliente": false,
    "flagErroReceitaFederal": false,
    "flagErroCredilink": false,
    "limitePoliticaConcessao": -1,
    "mensagemFiltro": "99. Accepted",
    "mensagemFiltroPolitica": "99. Accepted",
    "mensagemFiltroFinal": "99. Accepted",
    "solicitante": {
        "credilink": {
            "situacaoDocumento": "regular",
            "obito": "NAO"
        },
        "limites": {
            "limiteDisponivel": 0,
            "valorContratadoCartao": 0
        },
        "atrasos": {
            "qtdDiasAtrasoCartao": 0,
            "qtdDiasAtrasoParcelados": 0,
            "qtdDiasAtrasoBNPLMova": 0,
            "qtdDiasAtrasoP2PL": 0
        },
        "restritivosBVS": {
            "vlrMaxRestritivosBVSM1": 0,
            "vlrMaxRestritivosBVSM2": 0,
            "vlrMaxRestritivosBVSM3": 0
        },
        "idade": 30,
        "vlrMaxRestritivosSerasa90Dias": 0,
        "indFNV": false,
        "indReneg": false,
        "maxDiasAtraso": 0,
        "qtdIF": 1,
        "numeroDocumento": "12345678909",
        "dataAdmissaoFopag": "1001-01-01",
        "dataRegistroPrimeiroContrato": "1001-01-01",
        "dataRegistro": "2002-11-05T17:18:43.888Z",
        "saldoMinimo30d": 101,
        "flagFuncionario": 0,
        "flagMAT30d": 0,
        "flagMAT60d": 0,
        "flagMAT90d": 0,
        "fonteRenda": "01. Fopag",
        "flagPortabilidade": 0,
        "rendaLiquidaPicpay": 10000.0,
        "rendaLiquidaPicpayBatch": 10000.0,
        "consumerID": 1,
        "scoreAppSerasa": 999,
        "scoreInternoLongo": 999,
        "scoreInterno": 999,
        "investimentoTotal": 0.0,
        "saldoInvestimento": 0.0,
        "flagInvestimentoPrincipalidade": 1,
        "flagPrincipalidadeAll": 1,
        "flagPrincipalidade": 1,
        "mesFlagPrincipalidade": 3,
        "flagAllowlist": "-99999",
        "flagBlocklist": "-99999",
        "scoreInternoCurto": 999,
        "scoreExterno": 999,
        "flagCartaoCreditoContratado": 0,
        "flagLimiteGarantidoContratado": 0,
        "saldoMedio": 0.0,
        "flagCadastroIrregular": 0,
        "flagCadastroNegado": 0,
        "flagCadastroEmAndamento": 0,
        "flagContaInativa": 0,
        "flagContaTeste": 0,
        "flagOSDefasado": 0,
        "statusCPF": "REGULAR",
        "flagFraudePicpay": 0,
        "flagFraudeCartaoPicpay": 0,
        "flagHoldForced": 0,
        "flagChargebackPicpay": 0,
        "flagReneg": 0,
        "flagBloqueados": 0,
        "flagDesenrola": 0,
        "flagFNV": 0,
        "flagCartaoDebitoContratado": 0,
        "flagSmallLimits": 0,
        "mensagemFiltro": "99. Accepted",
        "flagCCRot": 0,
        "mediaSaldoTotal": 40000,
        "listaRestritivosSerasa": [
            {
                "tipo": "N",
                "montanteTotal": 0,
                "valorTotal": 0
            }
        ],
        "listaRestritivosBacen": [
            {
                "saldoVencido": 0,
                "prejuizo": 0,
                "creditoAVencerAdpos": 0,
                "creditoFinanciadoCartoes": 0,
                "creditoVencidoCartao": 0,
                "creditoAVencerCartao": 0,
                "limiteCartaoDisponivel": 0,
                "limiteChequeDisponivel": 0,
                "saldoBacenCheque": 0,
                "renegociacaoBacen": 0
            },
            {
                "saldoVencido": 0,
                "prejuizo": 0,
                "creditoAVencerAdpos": 0,
                "creditoFinanciadoCartoes": 0,
                "creditoVencidoCartao": 0,
                "creditoAVencerCartao": 0,
                "limiteCartaoDisponivel": 0,
                "limiteChequeDisponivel": 0,
                "saldoBacenCheque": 0,
                "renegociacaoBacen": 0
            },
            {
                "saldoVencido": 0,
                "prejuizo": 0,
                "creditoAVencerAdpos": 0,
                "creditoFinanciadoCartoes": 0,
                "creditoVencidoCartao": 0,
                "creditoAVencerCartao": 0,
                "limiteCartaoDisponivel": 0,
                "limiteChequeDisponivel": 0,
                "saldoBacenCheque": 0,
                "renegociacaoBacen": 0
            }
        ]
    },
    "payloadHomol": {
        "saidas": {
            "mensagemFiltro": "99. Accepted",
            "mensagemFinal": "99999. NAO PREENCHIDO",
            "statusDecisao": "NEGADO",
            "regrasNegativas": [
                {
                    "nomeRegra": "rl_limiteDisponivel",
                    "descricao": "34. limite dispon\u00edvel < = 0"
                }
            ],
            "listaLogs": [],
            "vlrLimiteAprovado": 0,
            "hasInvalidRegistration": 0,
            "hasDeniedOngoingRegistration": 0,
            "hasDeactivatedOrTestAccount": 0,
            "hasOutdatedOs": 0,
            "isInvalidCpf": 0,
            "isDeadUser": 0,
            "isFraudPP": 0,
            "isChargebackPP": 0,
            "isOutAgeRange": 0,
            "isDelayPP": 0,
            "isDelayOr": 0,
            "isRenegFNV": 0,
            "hasRestrictionBVS": 0,
            "hasRestrictionSerasa": 0,
            "isUserDesenrola": 0,
            "isNoHitBureau": 0,
            "etapa": "FIM",
            "flagTipoRenda": "BATCH"
        },
        "intermediarias": {
            "alavancagem": 3,
            "limitePiso": 2,
            "limiteTeto": 1,
            "limiteFixo": 0,
            "teste":4000000,
            "segmentacao":"teste",
            "aprovadoSLAux":"nomeBlaze"
        }
    },
    "idSolicitacao": 14
}
'''



json_object = json.loads(jsonAnalise)

In [11]:
counter = 0
for i in json_object["payloadHomol"]["saidas"]:
    print(i)
    counter+=1
print(counter)

mensagemFiltro
mensagemFinal
statusDecisao
regrasNegativas
listaLogs
vlrLimiteAprovado
hasInvalidRegistration
hasDeniedOngoingRegistration
hasDeactivatedOrTestAccount
hasOutdatedOs
isInvalidCpf
isDeadUser
isFraudPP
isChargebackPP
isOutAgeRange
isDelayPP
isDelayOr
isRenegFNV
hasRestrictionBVS
hasRestrictionSerasa
isUserDesenrola
isNoHitBureau
etapa
flagTipoRenda
24


In [12]:
counter = 0
for i in json_object["payloadHomol"]['saidas']["listaLogs"]:
    print(i)
    counter+=1
print(counter)

0


In [14]:
def addListaLogs(payload):
    intermediarias = payload['payloadHomol']['intermediarias']
    listaLogs = []

    DictIntermediarias = {
        "aprovadoBAU": intermediarias.get("aprovadoBAUAux", None),
        "aprovadoSmallLimits": intermediarias.get("aprovadoSLAux", None),
        "clienteExistente": intermediarias.get("flagClienteExistente", None),
        "dataAdmissao": intermediarias.get("dataAdmissao", None),
        "dataAdmissaoMeses": intermediarias.get("admissaoMeses", None),
        "clienteMAT": intermediarias.get("clienteMAT", None),
        "fonteRenda": intermediarias.get("flagTipoRenda", None),
        "rendaInexistente": intermediarias.get("rendaInexistente", None),
        "clienteNovo": intermediarias.get("flagClienteNovo", None),
        "naoCliente": intermediarias.get("flagNaoCliente", None),
        "scoreInexistente": intermediarias.get("semInfoScore", None),
        "principalidadeInvestimento": intermediarias.get("flagPrincipalidade", None),
        "principalidadeAll": intermediarias.get("principalidadeAll", None),
        "principalidadeAllRecorrencia": intermediarias.get("recorrenciaPrincipalidade", None),
        "clienteBlindado": intermediarias.get("clienteBlindado", None),
        "clienteBloqueado": intermediarias.get("clienteBloqueado", None),
        "faixaRendaLiquida": intermediarias.get("faixaRendaLiquida", None),
        "faixaRendaBruta": intermediarias.get("faixaRendaBruta", None),
        "faixaScoreAppcardBlend": intermediarias.get("faixaScoreInternoCurto", None),
        "faixaScoreExterno": intermediarias.get("faixaScoreExterno", None),
        "segmentacaoNegocio": intermediarias.get("segmentacaoNegocio", None),
        "segmentacaoPolitica": intermediarias.get("segmentacaoPolitica", None),
        "rangeScoreInterno": intermediarias.get("faixaRiscoInternoAgrupada", None),
        "segmentacaoSubgrupo": intermediarias.get("segmentacao", None),
        "aprovadoBAUAux": intermediarias.get("aprovadoBAUAux", None),
        "aprovadoSmallLimitsAux": intermediarias.get("aprovadoSLAux", None),
        "limiteFixo": intermediarias.get("limiteFixo", None),
        "alavancagem": intermediarias.get("alavancagem", None),
        "limiteTeto": intermediarias.get("limiteTeto", None),
        "limitePiso": intermediarias.get("limitePiso", None)
    }

    varsTemp = [listaLogs.append({"nome": key, "valor": value}) for key, value in intermediarias.items() if key in DictIntermediarias.keys()]

    payload['payloadHomol']['saidas']['listaLogs'] = listaLogs

    return payload


addListaLogs(json_object)



{'marcacaoMesa': 'N',
 'etapa': 'FILTROS POLITICA',
 'flagErroBacen': False,
 'flagErroSerasa': False,
 'flagErroAurora': False,
 'flagErroVisaoCliente': False,
 'flagErroReceitaFederal': False,
 'flagErroCredilink': False,
 'limitePoliticaConcessao': -1,
 'mensagemFiltro': '99. Accepted',
 'mensagemFiltroPolitica': '99. Accepted',
 'mensagemFiltroFinal': '99. Accepted',
 'solicitante': {'credilink': {'situacaoDocumento': 'regular', 'obito': 'NAO'},
  'limites': {'limiteDisponivel': 0, 'valorContratadoCartao': 0},
  'atrasos': {'qtdDiasAtrasoCartao': 0,
   'qtdDiasAtrasoParcelados': 0,
   'qtdDiasAtrasoBNPLMova': 0,
   'qtdDiasAtrasoP2PL': 0},
  'restritivosBVS': {'vlrMaxRestritivosBVSM1': 0,
   'vlrMaxRestritivosBVSM2': 0,
   'vlrMaxRestritivosBVSM3': 0},
  'idade': 30,
  'vlrMaxRestritivosSerasa90Dias': 0,
  'indFNV': False,
  'indReneg': False,
  'maxDiasAtraso': 0,
  'qtdIF': 1,
  'numeroDocumento': '12345678909',
  'dataAdmissaoFopag': '1001-01-01',
  'dataRegistroPrimeiroContrato

In [2]:
def addListaLogs(payload):
    intermediarias = payload['payloadHomol']['intermediarias']
    ListaLogs = []

    DictIntermediarias = {
        "aprovadoBAU": intermediarias.get("aprovadoBAUAux", None),
        "aprovadoSmallLimits": intermediarias.get("aprovadoSLAux", None),
        "clienteExistente": intermediarias.get("flagClienteExistente", None),
        "dataAdmissao": intermediarias.get("dataAdmissao", None),
        "dataAdmissaoMeses": intermediarias.get("admissaoMeses", None),
        "clienteMAT": intermediarias.get("clienteMAT", None),
        "fonteRenda": intermediarias.get("flagTipoRenda", None),
        "rendaInexistente": intermediarias.get("rendaInexistente", None),
        "clienteNovo": intermediarias.get("flagClienteNovo", None),
        "naoCliente": intermediarias.get("flagNaoCliente", None),
        "scoreInexistente": intermediarias.get("semInfoScore", None),
        "principalidadeInvestimento": intermediarias.get("flagPrincipalidade", None),
        "principalidadeAll": intermediarias.get("principalidadeAll", None),
        "principalidadeAllRecorrencia": intermediarias.get("recorrenciaPrincipalidade", None),
        "clienteBlindado": intermediarias.get("clienteBlindado", None),
        "clienteBloqueado": intermediarias.get("clienteBloqueado", None),
        "faixaRendaLiquida": intermediarias.get("faixaRendaLiquida", None),
        "faixaRendaBruta": intermediarias.get("faixaRendaBruta", None),
        "faixaScoreAppcardBlend": intermediarias.get("faixaScoreInternoCurto", None),
        "faixaScoreExterno": intermediarias.get("faixaScoreExterno", None),
        "segmentacaoNegocio": intermediarias.get("segmentacaoNegocio", None),
        "segmentacaoPolitica": intermediarias.get("segmentacaoPolitica", None),
        "rangeScoreInterno": intermediarias.get("faixaRiscoInternoAgrupada", None),
        "segmentacaoSubgrupo": intermediarias.get("segmentacao", None),
        "aprovadoBAUAux": intermediarias.get("aprovadoBAUAux", None),
        "aprovadoSmallLimitsAux": intermediarias.get("aprovadoSLAux", None),
        "limiteFixo": intermediarias.get("limiteFixo", None),
        "alavancagem": intermediarias.get("alavancagem", None),
        "limiteTeto": intermediarias.get("limiteTeto", None),
        "limitePiso": intermediarias.get("limitePiso", None)
    }

    # for key, value in DictIntermediarias.items():
    #     if value is not None:
    #         ListaLogs.append({"nome": key, "valor": value})
    
            
    varTemp = [ListaLogs.append({"nome": key, "valor": value}) for key, value in DictIntermediarias.items() if value is not None ]

    payload['payloadHomol']['ListaLogs'] = ListaLogs

    return payload


addListaLogs(json_object)

{'marcacaoMesa': 'N',
 'etapa': 'FILTROS POLITICA',
 'flagErroBacen': False,
 'flagErroSerasa': False,
 'flagErroAurora': False,
 'flagErroVisaoCliente': False,
 'flagErroReceitaFederal': False,
 'flagErroCredilink': False,
 'limitePoliticaConcessao': -1,
 'mensagemFiltro': '99. Accepted',
 'mensagemFiltroPolitica': '99. Accepted',
 'mensagemFiltroFinal': '99. Accepted',
 'solicitante': {'credilink': {'situacaoDocumento': 'regular', 'obito': 'NAO'},
  'limites': {'limiteDisponivel': 0, 'valorContratadoCartao': 0},
  'atrasos': {'qtdDiasAtrasoCartao': 0,
   'qtdDiasAtrasoParcelados': 0,
   'qtdDiasAtrasoBNPLMova': 0,
   'qtdDiasAtrasoP2PL': 0},
  'restritivosBVS': {'vlrMaxRestritivosBVSM1': 0,
   'vlrMaxRestritivosBVSM2': 0,
   'vlrMaxRestritivosBVSM3': 0},
  'idade': 30,
  'vlrMaxRestritivosSerasa90Dias': 0,
  'indFNV': False,
  'indReneg': False,
  'maxDiasAtraso': 0,
  'qtdIF': 1,
  'numeroDocumento': '12345678909',
  'dataAdmissaoFopag': '1001-01-01',
  'dataRegistroPrimeiroContrato

In [3]:
teste = json.dumps(json_object,indent=4)

print(teste)

{
    "marcacaoMesa": "N",
    "etapa": "FILTROS POLITICA",
    "flagErroBacen": false,
    "flagErroSerasa": false,
    "flagErroAurora": false,
    "flagErroVisaoCliente": false,
    "flagErroReceitaFederal": false,
    "flagErroCredilink": false,
    "limitePoliticaConcessao": -1,
    "mensagemFiltro": "99. Accepted",
    "mensagemFiltroPolitica": "99. Accepted",
    "mensagemFiltroFinal": "99. Accepted",
    "solicitante": {
        "credilink": {
            "situacaoDocumento": "regular",
            "obito": "NAO"
        },
        "limites": {
            "limiteDisponivel": 0,
            "valorContratadoCartao": 0
        },
        "atrasos": {
            "qtdDiasAtrasoCartao": 0,
            "qtdDiasAtrasoParcelados": 0,
            "qtdDiasAtrasoBNPLMova": 0,
            "qtdDiasAtrasoP2PL": 0
        },
        "restritivosBVS": {
            "vlrMaxRestritivosBVSM1": 0,
            "vlrMaxRestritivosBVSM2": 0,
            "vlrMaxRestritivosBVSM3": 0
        },
        

In [3]:
import pandas as pd 

tabelaJaCliente = pd.read_csv('C:/Users/T51657/Desktop/Homologação/Modelos Online/v2/arquivos/tabelasHomologacao/Ja_Cliente_Segmentacao_codigo_motor.csv')

In [5]:
from pyspark.sql import SparkSession


spark = SparkSession.builder.appName('abc').getOrCreate()
sparkDF=spark.createDataFrame(tabelaJaCliente) 
sparkDF.printSchema()
sparkDF.show()


root
 |-- SEGMENTAÇÃO FINAL: string (nullable = true)
 |-- rangeScoreInterno: string (nullable = true)
 |-- segmentacaoPolitica: string (nullable = true)
 |-- faixaRendaBruta: string (nullable = true)
 |-- flagFuncionario: long (nullable = true)
 |-- FLAG_CC_ROT: string (nullable = true)
 |-- mean_total_balance_d0: string (nullable = true)
 |-- cpf6e7Digito: string (nullable = true)
 |-- isApprovedBauAux: long (nullable = true)
 |-- isApprovedSmallLimits: long (nullable = true)
 |-- limiteFixo: long (nullable = true)
 |-- alavancagem: double (nullable = true)
 |-- limiteTeto: long (nullable = true)
 |-- limitePiso: long (nullable = true)
 |-- cenario: long (nullable = true)

+-----------------+-----------------+-------------------+---------------+---------------+-----------+---------------------+------------+----------------+---------------------+----------+-----------+----------+----------+-------+
|SEGMENTAÇÃO FINAL|rangeScoreInterno|segmentacaoPolitica|faixaRendaBruta|flagFuncionari

In [10]:

from pyspark.sql.functions import col

teste = sparkDF.select('cpf6e7Digito').filter(col('rangeScoreInterno' == "R2")).distinct()

Py4JError: An error occurred while calling z:org.apache.spark.sql.functions.col. Trace:
py4j.Py4JException: Method col([class java.lang.Boolean]) does not exist
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:321)
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:342)
	at py4j.Gateway.invoke(Gateway.java:276)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.ClientServerConnection.waitForCommands(ClientServerConnection.java:182)
	at py4j.ClientServerConnection.run(ClientServerConnection.java:106)
	at java.base/java.lang.Thread.run(Thread.java:834)

